# Byte Pair Encoding (Tutorial 0x01) - The Algorithm
(C) Maxim Gansert, 2019, 2022, Mindscan Engineering


Byte Pair Encoding is a compression scheme, (I found a description of that in an article in the "C/C++ Users Journal" of September 1997, Page 23--30, by Philip Gage)

Today this algorithm is used for encoding textual inputs, which is preprocessed for NLP/NLU-tasks (natural language processing, natural language understanding) it allows encoding/pairing the characters in a sub-word-level and also provides an option for encoding unknown words.

How does it being intended to work?

### Encode ###


Original Description says:

The following pseudocode illustrates the compression process:
~~~~
Read file into buffer
While (compression possible)
{
    find most frequent byte pair in buffer
    
    add pair to table and assign it an unused byte
    
    replace all such pairs with this unused byte
}
Write pair table and buffer
~~~~

### Decode ###

Original Description says:

The following pseudocode illustrates the decompression process:

~~~~
read and store the pair table in buffer
while ( stack not empty or not end of file)
{
    if (stack empty) read byte
    else pop byte from stack
    
    if(byte is pair code) push pair on stack
    else write byte      
}
~~~~

### Process ###

After each compression step the data to compress needs a reanalysis. So this is a multipass step.

# Byte Pair Encoding (Tutorial 0x02) - Source Code Tokenization

Actually this Encoding is somehow more universal. Today it is used in Language Models for tokenizing on a subword level.

Taking this idea, it allows us to understand source code changes, better and mark differences between two lines not on a character by character level but on a level on meaning, because we can use statistics to figure out which parts probably should be considered together.

It allows for a quite good and human oriented tokenization. This is an idea, which tries to calculate the difference between two lines of code. But also understand the difference better, instead of comming up with clever heuristics for the character level. This seems to simplify the diff problem by a lot, and leads to a tokenization based on source code statistics.

See the notebooks in the bpe_diff folder. I haven't found any reference to the idea to tokenize a diff and work with these tokens to calculate a better human readable diff from it.

# Byte Pair Encoding (Tutorial 0x03) - Reuse Statistics

Als a data compression technique this algorithm is quite slow. The idea is to combine the two most adjacently occuring tokens into a new one. So if the 'e' and 'r' token are occuring very often together, and more often than any other we combine these two into a new token 'er' and now find out the now most occuring combination and might find 'i','n'.

The main idea is to keep that statistics in what particular order we combined two tokens. If you do this for a large enough corpus we now know a ruleset in which natural order to combine tokens. We can aim for a dictionary size like 10K tokens, that means we store the 10000 most together occuring tokens, where many of the tokens represent full words. If we reapply the same ruleset, which we calculated across a large corpus of text or source code, the result of the encoding operation canbe speed up, instead of counting token pairs, and then selecting one of them as the most orring one, we just assume we calculated th is statistics and we "just know better" instead of searching for most effective solution.

This ruleset resuse allows us now to encode two files, with the same ruleset, where the same words basically lead to the same combined token. But this generalized ruleset also contains a lot of knowledge about which characters belong to eah other and which are more unlikely to occur together - which can be understood as a divider for words.

This rulreset reuse allows now to tokenize the same way and without the recalculation of statistics, we just spend time to figure out which tokens to combine first, then second, then third.

We start with tokens being one single letter and ending combining 'artifact' with 's' (but this depends very much on the corpus used to obtain these statistics). 

Anyways this compression of the whole corpus leads to aquite useful knowledge about which letters belong together and which don't. This in itself basically is alreadyy a language model on it's own. Because of the now fixed ruleset, we will always encode/transform the same word in the same form and arrive at the same final token for that word. This property makes it useful for another language model.